# Hochschule Bonn-Rhein-Sieg

# Probabilistic Methods for Robotics, WS21

# Assignment 07

Instructions for submission :
- Please restart and run all cells before submitting 
- Make sure your user name is correct

Good luck !!

In [1]:
import numpy as np
import IPython
import matplotlib.pyplot as plt

### Student user name: nmursa2s


### Exercise 1: First- and second-order Markov process [20 points]

Show that any second-order Markov process can be rewritten as a first-order Markov
process with an augmented set of state variables. Can this always be done *parsimoniously*,
i.e., without increasing the number of parameters needed to specify the transition model?

We know that First-order Markov process says that current state depends only on the previous state and not on any earlier states

$$ P(X_t | X_{0 : t-1}) = P(X_t | X_{t-1})$$

Markov chains with higher order are the processes in which the next state depends on two or more preceding ones. So, in second-order we have the states depend on two preceding ones. 

There is nothing radically different about second order Markov chains: if $$P(x_i|x_{i-1},..,x_1)=P(x_i|x_{i-1},..,x_{i-n})$$ is a "n-th order Markov chain", it is also possible to write it as a first order Markov chain, on the space of combinations of $n$ states, For example, $S^n$, if $S$ is the combination/set of all values $x_i$ has: We can just write $$P(x_i|x_{i-1},..,x_1)=P(x_i|x_{i-1},..,x_{i-n})=P((x_i,x_{i-1},..,x_{i-n+1})|(x_{i-1},x_{i-2},..,x_{i-n}))$$

Thus a second-order markov chain is just one which takes into account the two previous states. 

### Exercise 2: Speech recognition using the Viterbi algorithm [80 points]

In assignment 1, we considered a simple speech synthesis system that generates sequences of the letters $\{$ a, b, c, d $\}$; now we are finally ready to examine that problem a little bit. In particular, we are going to start examining the Viterbi algorithm, which will be covered in the lectures in the following weeks.

In assignment 1, we said that that the prior probabilities of generating the letters are $P(a) = 0.1$, $P(b) = 0.4$, $P(c) = 0.2$, and $P(d) = 0.3$; in addition, we assumed that we are given the following probabilities of generating letter $n+1$ given letter $n$:

$P(a|a) = 0.2 \hspace{2cm} P(b|a) = 0.1 \hspace{2cm} P(c|a) = 0.6 \hspace{2cm} P(d|a) = 0.1$

$P(a|b) = 0.4 \hspace{2cm} P(b|b) = 0.2 \hspace{2cm} P(c|b) = 0.1 \hspace{2cm} P(d|b) = 0.3$

$P(a|c) = 0.1 \hspace{2cm} P(b|c) = 0.2 \hspace{2cm} P(c|c) = 0.4 \hspace{2cm} P(d|c) = 0.3$

$P(a|d) = 0.4 \hspace{2cm} P(b|d) = 0.4 \hspace{2cm} P(c|d) = 0.2 \hspace{2cm} P(d|d) = 0.0$

These are our *state transition probabilities*. We assume that a letter is correctly observed with $0.97$ probability; this means that there is $0.03$ probability that a letter will be observed incorrectly, a probability that is uniformly distributed over the other three letters. Given this information, we want to find answers to the following questions:

1. Let's assume that we have observed the sequence $ac$; what is the most likely sequence that has generated this observation?
2. We now observe the sequence $bbcdd$; what is the most likely sequence that has generated this observation?

Your task is to implement the function *most_likely_sequence* in the cell below; this function should use the Viterbi algorithm for finding the most likely sequences that have generated the observed ones.

In [2]:
def most_likely_sequence(observed_sequence: list, letters, priors, \
                         transition_probabilities, observation_probabilities):
    max_sequence = list()
    ### write your code here ###
    total_states = len(observed_sequence)
    viterbi = np.zeros((len(letters), len(observed_sequence)))
    pointers = np.zeros((len(letters), len(observed_sequence)))
    #viterbi = np.array([priors[i] * observation_probabilities[i] for i in range(0,len(letters))])
    #pointers = np.array([priors[i] * observation_probabilities[i] for i in range(0,len(observed_sequence))])
    # we need to fill values for observation 1 from the starting state s to number of states (in our case it is total_state)
    for s in range(len(letters)): # we iterate through number of states 
        viterbi[s][0] = priors[s] * observation_probabilities[s][0]
        pointers[s][0] = 0  
    argmax = 0

    print(viterbi)
    print(viterbi.shape)
    for o in range(1, total_states): # we iterate through t is observation time, 
        for s in range(len(letters)): # we iterate through s is our state
            max = 0
            argmax = 0
            for i in range(1, total_states): # i iterates through number of all states 
                if max < viterbi[i][o - 1] * transition_probabilities[i][s] * observation_probabilities[s][o]:
                    max = viterbi[i][o - 1] * transition_probabilities[i][s] * observation_probabilities[s][o]
                    argmax = i
            viterbi[s][o] = max # our trellis is equal to max
            pointers[s][o] = argmax #pointers is equal to k which is argmax in our case

    for i in range(0, total_states):
        max_sequence.append(0)
    max = 0
    for i in range(1,total_states):
        if max < viterbi[i][total_states- 1]:
            max = viterbi[i][total_states - 1]
            argmax = i
            
    max_sequence[total_states - 1] = argmax

    for i in range(total_states - 1, 1, -1): ## Backtrack from last observation.
        max_sequence[i - 1] = pointers[max_sequence[i]][i] ## Insert previous state on most likely path
    for i in range(1, len(max_sequence)):
        max_sequence.append(letters[max_sequence[i]])
    ### your code ends here ###
    return max_sequence

### Testing your code

The cell below describes the above problem and calls the *most_likely_sequence* function with the sequences $ac$ and $bbcdd$. Please run the cell once you're done with your implementation; you should obtain $ac$ and $bbcda$ as the most likely sequences corresponding to the observed ones.

In [3]:
###sample result
#1)['a', 'c']->[]
#2)['b', 'b', 'c', 'd', 'd']->[]

letters = ['a', 'b', 'c', 'd']
priors = np.array([0.1, 0.4, 0.2, 0.3])
transition_probabilities = np.array([[0.2, 0.4, 0.1, 0.4],\
                                     [0.1, 0.2, 0.2, 0.4],\
                                     [0.6, 0.1, 0.4, 0.2],\
                                     [0.1, 0.3, 0.3, 0.0]])
observation_probabilities = .01 * np.ones((4,4)) #so our observation probabilities are 4x4 array
np.fill_diagonal(observation_probabilities, np.array([0.97, 0.97, 0.97, 0.97]))

observation1 = ['a', 'c']
max_sequence = most_likely_sequence(observation1, letters, priors, \
                                    transition_probabilities, observation_probabilities)
print(str(observation1) +  '->' + str(max_sequence))

observation2 = ['b', 'b', 'c', 'd', 'd']
max_sequence = most_likely_sequence(observation2, letters, priors, \
                                    transition_probabilities, observation_probabilities)
print(str(observation2) + '->' + str(max_sequence))

[[0.097 0.   ]
 [0.004 0.   ]
 [0.002 0.   ]
 [0.003 0.   ]]
(4, 2)
['a', 'c']->[0, 1, 'b']
[[0.097 0.    0.    0.    0.   ]
 [0.004 0.    0.    0.    0.   ]
 [0.002 0.    0.    0.    0.   ]
 [0.003 0.    0.    0.    0.   ]]
(4, 5)


IndexError: index 4 is out of bounds for axis 0 with size 4